# Đồ án 3: Linear Regression

## Import thư viện

In [ ]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import KFold

## Linear Regresstion

In [ ]:
def LinearRegression(A, b):
    reg = linear_model.LinearRegression().fit(A, b)
    return reg.coef_, reg.intercept_

## Cross Validation

In [ ]:
def CrossValidation(A, b, kfold):
    kf = KFold(n_splits=kfold)
    error_list = []

    for train_index, test_index in kf.split(A):
        A_train, A_test = A[train_index], A[test_index]
        b_train, b_test = b[train_index], b[test_index]
        x, bias = LinearRegression(A_train, b_train)
        error_list.append(np.mean(np.absolute(b_test - (np.dot(A_test, x) + bias))))

    return np.mean(error_list)

## Chọn tính chất tốt nhất

In [ ]:
def chooseBestProperty(A, b):
    result = []

    for i in range(A.shape[1]):
        cv_prop = CrossValidation(A[:, i:i + 1], b, 10)
        result.append(cv_prop)
    
    return result, np.argmin(result)

## Xây dựng model (Chọn top các tính chất để xây dựng)

In [ ]:
def buildModel(A, b, error_list, label_len):
    sorted_error = np.argsort(error_list)
    full_error = CrossValidation(A, b, 10)

    error_list = []
    cv_error_list = []
    prop_list = []

    for i in range(2, label_len - 1):
        prop_to_pick = sorted_error[:i]
        A_build = A[:, prop_to_pick]
        error = CrossValidation(A_build, b, 10)
        cv_error_list.append(error)
        error_list.append(np.absolute(full_error - error))
        prop_list.append(prop_to_pick)

    return cv_error_list, prop_list, np.argmin(error_list)

## Test functions

### Đọc và tách dữ liệu

In [ ]:
df = pd.read_csv('wine.csv', sep=';')

In [ ]:
label = df.columns
properties = df.iloc[:, : -1].to_numpy()
rate = df.iloc[:, -1].to_numpy()

### Câu a: Xây dựng mô hình trên tất cả các tính chất

In [ ]:
# Xây dựng mô hình
x_a, bias_a = LinearRegression(properties, rate)

# Tính sai số của mô hình
error_a = CrossValidation(properties, rate, 10)

In [ ]:
print(f'Model: A{x_a} + {bias_a} = b')
print(f'CV error: {error_a}')

### Câu b: Chọn tính chất tốt nhất và xây dựng mô hình trên tính chất đó

In [ ]:
# Chọn tính chất tốt nhất
error_list, best_index = chooseBestProperty(properties, rate)

# Xây dựng mô hình
x_b, bias_b = LinearRegression(properties[:, best_index:best_index + 1], rate)

In [ ]:
print(f'Best property is {label[best_index]}')
print(f'Model: A{x_b} + {bias_b} = b')
print(f'CV error: {error_list[best_index]}')

## Câu c: Xây dựng mô hình riêng

In [ ]:
# Chọn top x tính chất tốt nhất (x nào là tốt nhất)
cv_error_list, prop_list, best_index_prop = buildModel(properties, rate, error_list, len(label))

# Xây dựng mô hình
x_c, bias_c = LinearRegression(properties[:, prop_list[best_index_prop]], rate)

In [ ]:
print(f'Best properties are: {list(label[prop_list[best_index_prop]])}')
print(f'Model: A{x_c} + {bias_c} = b')
print(f'CV error: {cv_error_list[best_index_prop]}')